In [ ]:
import numpy as np 
import pandas as pd 
import faiss
from sentence_transformers import SentenceTransformer
import warnings
import math

In [ ]:

warnings.simplefilter('ignore')
pd.set_option("max_colwidth", None)
pd.options.display.float_format = '{:.2f}'.format


In [15]:
model = SentenceTransformer('bert-base-nli-mean-tokens')

In [16]:
movies=pd.read_csv("./tmdb_5000_movies.csv")
movies.shape
movies.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {""id"": 878, ""name"": ""Science Fiction""}]",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""space war""}, {""id"": 3388, ""name"": ""space colony""}, {""id"": 3679, ""name"": ""society""}, {""id"": 3801, ""name"": ""space travel""}, {""id"": 9685, ""name"": ""futuristic""}, {""id"": 9840, ""name"": ""romance""}, {""id"": 9882, ""name"": ""space""}, {""id"": 9951, ""name"": ""alien""}, {""id"": 10148, ""name"": ""tribe""}, {""id"": 10158, ""name"": ""alien planet""}, {""id"": 10987, ""name"": ""cgi""}, {""id"": 11399, ""name"": ""marine""}, {""id"": 13065, ""name"": ""soldier""}, {""id"": 14643, ""name"": ""battle""}, {""id"": 14720, ""name"": ""love affair""}, {""id"": 165431, ""name"": ""anti war""}, {""id"": 193554, ""name"": ""power relations""}, {""id"": 206690, ""name"": ""mind and soul""}, {""id"": 209714, ""name"": ""3d""}]",en,Avatar,"In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.",150.44,"[{""name"": ""Ingenious Film Partners"", ""id"": 289}, {""name"": ""Twentieth Century Fox Film Corporation"", ""id"": 306}, {""name"": ""Dune Entertainment"", ""id"": 444}, {""name"": ""Lightstorm Entertainment"", ""id"": 574}]","[{""iso_3166_1"": ""US"", ""name"": ""United States of America""}, {""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""}]",2009-12-10,2787965087,162.00,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,Enter the World of Pandora.,Avatar,7.20,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {""id"": 28, ""name"": ""Action""}]",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""name"": ""drug abuse""}, {""id"": 911, ""name"": ""exotic island""}, {""id"": 1319, ""name"": ""east india trading company""}, {""id"": 2038, ""name"": ""love of one's life""}, {""id"": 2052, ""name"": ""traitor""}, {""id"": 2580, ""name"": ""shipwreck""}, {""id"": 2660, ""name"": ""strong woman""}, {""id"": 3799, ""name"": ""ship""}, {""id"": 5740, ""name"": ""alliance""}, {""id"": 5941, ""name"": ""calypso""}, {""id"": 6155, ""name"": ""afterlife""}, {""id"": 6211, ""name"": ""fighter""}, {""id"": 12988, ""name"": ""pirate""}, {""id"": 157186, ""name"": ""swashbuckler""}, {""id"": 179430, ""name"": ""aftercreditsstinger""}]",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, has come back to life and is headed to the edge of the Earth with Will Turner and Elizabeth Swann. But nothing is quite as it seems.",139.08,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""name"": ""Jerry Bruckheimer Films"", ""id"": 130}, {""name"": ""Second Mate Productions"", ""id"": 19936}]","[{""iso_3166_1"": ""US"", ""name"": ""United States of America""}]",2007-05-19,961000000,169.00,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.90,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 80, ""name"": ""Crime""}]",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name"": ""based on novel""}, {""id"": 4289, ""name"": ""secret agent""}, {""id"": 9663, ""name"": ""sequel""}, {""id"": 14555, ""name"": ""mi6""}, {""id"": 156095, ""name"": ""british secret service""}, {""id"": 158431, ""name"

In [17]:
movies=movies[pd.notnull(movies['overview'])]
movies=movies.reset_index(drop=True)
movies['id']=movies.index

movies.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {""id"": 878, ""name"": ""Science Fiction""}]",http://www.avatarmovie.com/,0,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""space war""}, {""id"": 3388, ""name"": ""space colony""}, {""id"": 3679, ""name"": ""society""}, {""id"": 3801, ""name"": ""space travel""}, {""id"": 9685, ""name"": ""futuristic""}, {""id"": 9840, ""name"": ""romance""}, {""id"": 9882, ""name"": ""space""}, {""id"": 9951, ""name"": ""alien""}, {""id"": 10148, ""name"": ""tribe""}, {""id"": 10158, ""name"": ""alien planet""}, {""id"": 10987, ""name"": ""cgi""}, {""id"": 11399, ""name"": ""marine""}, {""id"": 13065, ""name"": ""soldier""}, {""id"": 14643, ""name"": ""battle""}, {""id"": 14720, ""name"": ""love affair""}, {""id"": 165431, ""name"": ""anti war""}, {""id"": 193554, ""name"": ""power relations""}, {""id"": 206690, ""name"": ""mind and soul""}, {""id"": 209714, ""name"": ""3d""}]",en,Avatar,"In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.",150.44,"[{""name"": ""Ingenious Film Partners"", ""id"": 289}, {""name"": ""Twentieth Century Fox Film Corporation"", ""id"": 306}, {""name"": ""Dune Entertainment"", ""id"": 444}, {""name"": ""Lightstorm Entertainment"", ""id"": 574}]","[{""iso_3166_1"": ""US"", ""name"": ""United States of America""}, {""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""}]",2009-12-10,2787965087,162.00,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,Enter the World of Pandora.,Avatar,7.20,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {""id"": 28, ""name"": ""Action""}]",http://disney.go.com/disneypictures/pirates/,1,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""name"": ""drug abuse""}, {""id"": 911, ""name"": ""exotic island""}, {""id"": 1319, ""name"": ""east india trading company""}, {""id"": 2038, ""name"": ""love of one's life""}, {""id"": 2052, ""name"": ""traitor""}, {""id"": 2580, ""name"": ""shipwreck""}, {""id"": 2660, ""name"": ""strong woman""}, {""id"": 3799, ""name"": ""ship""}, {""id"": 5740, ""name"": ""alliance""}, {""id"": 5941, ""name"": ""calypso""}, {""id"": 6155, ""name"": ""afterlife""}, {""id"": 6211, ""name"": ""fighter""}, {""id"": 12988, ""name"": ""pirate""}, {""id"": 157186, ""name"": ""swashbuckler""}, {""id"": 179430, ""name"": ""aftercreditsstinger""}]",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, has come back to life and is headed to the edge of the Earth with Will Turner and Elizabeth Swann. But nothing is quite as it seems.",139.08,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""name"": ""Jerry Bruckheimer Films"", ""id"": 130}, {""name"": ""Second Mate Productions"", ""id"": 19936}]","[{""iso_3166_1"": ""US"", ""name"": ""United States of America""}]",2007-05-19,961000000,169.00,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.90,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 80, ""name"": ""Crime""}]",http://www.sonypictures.com/movies/spectre/,2,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name"": ""based on novel""}, {""id"": 4289, ""name"": ""secret agent""}, {""id"": 9663, ""name"": ""sequel""}, {""id"": 14555, ""name"": ""mi6""}, {""id"": 156095, ""name"": ""british secret service""}, {""id"": 158431, ""name"": ""united

In [ ]:
sentences=movies['overview'].tolist()
print("Number of Sentences in Movie overview ",len(sentences))

Number of Sentences in Movie Description  4800


In [ ]:
embeddings=model.encode(sentences)
faiss.normalize_L2(embeddings) 


In [20]:
print("Shape of the EMbeddings is ",embeddings.shape)

Shape of the EMbeddings is  (4800, 768)


In [ ]:
dim=768
ncentroids=50 ### This is a hyperparameter, and indicates number of clusters to be split into
m=16 
quantiser = faiss.IndexFlatL2(dim)
index = faiss.IndexIVFPQ (quantiser, dim,ncentroids, m , 8)
index.train(embeddings) 
print(index.is_trained)
faiss.write_index(index, "trained.index")


True


In [22]:
ids=movies['id'].tolist()
ids=np.array(ids)
index.add_with_ids(embeddings,ids)
print(index.ntotal)

4800


In [23]:
faiss.write_index(index,"block.index")

In [24]:
def calculateInnerProduct(L2_score):
    return (2-math.pow(L2_score,2))/2

In [25]:
def searchFAISSIndex(data,id_col_name,query,index,nprobe,model,topk=20):
    ## Convert the query into embeddings
    query_embedding=model.encode([query])[0]
    dim=query_embedding.shape[0]
    query_embedding=query_embedding.reshape(1,dim)
    faiss.normalize_L2(query_embedding)
  
    
    index.nprobe=nprobe
    
    D,I=index.search(query_embedding,topk) 
    ids=[i for i in I][0]
    L2_score=[d for d in D][0]
    inner_product=[calculateInnerProduct(l2) for l2 in L2_score]
    search_result=pd.DataFrame()
    search_result[id_col_name]=ids
    search_result['cosine_sim']=inner_product
    search_result['L2_score']=L2_score
    dat=data[data[id_col_name].isin(ids)]
    dat=pd.merge(dat,search_result,on=id_col_name)
    dat=dat.sort_values('cosine_sim',ascending=False)
    return dat

In [ ]:
query=""
search_result=searchFAISSIndex(movies,"id",query,index,nprobe=10,model=model,topk=10)
search_result=search_result[['id','overview','title','cosine_sim','L2_score', 'genres']]
search_result

,id,overview,title,cosine_sim,L2_score,genres
0,152,"Continuing his ""legendary adventures of awesomeness"", Po must face two hugely epic, but different threats: one supernatural and the other a little closer to his home.",Kung Fu Panda 3,0.65,0.84,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 16, ""name"": ""Animation""}, {""id"": 35, ""name"": ""Comedy""}, {""id"": 10751, ""name"": ""Family""}]"
1,465,"Blending lively music and brilliant animation, this sequel to the original 'Fantasia' restores 'The Sorcerer's Apprentice' and adds seven new shorts.",Fantasia 2000,0.63,0.86,"[{""id"": 16, ""name"": ""Animation""}, {""id"": 10751, ""name"": ""Family""}, {""id"": 10402, ""name"": ""Music""}]"
4,3978,Some of the world's most innovative documentary filmmakers will explore the hidden side of everything.,Freakonomics,0.62,0.87,"[{""id"": 99, ""name"": ""Documentary""}]"
9,4531,The life and career of renowned magician and sleight of hand artist Ricky Jay.,Deceptive Practice: The Mysteries and Mentors of Ricky Jay,0.61,0.89,"[{""id"": 99, ""name"": ""Documentary""}]"
5,4049,"Walt Disney's timeless masterpiece is an extravaganza of sight and sound! See the music come to life, hear the pictures burst into song and experience the excitement that is Fantasia over and over again.",Fantasia,0.60,0.89,"[{""id"": 16, ""name"": ""Animation""}, {""id"": 10751, ""name"": ""Family""}, {""id"": 10402, ""name"": ""Music""}]"
3,3958,"This film is a spoof along the lines of ""Scary Movie"" and ""Not Another Teen Movie."" It includes parodies of some of the indie film world's most renowned movies such as ""Memento,"" ""Pulp Fiction,"" ""Magnolia,"" ""My Big Fat Greek Wedding,"" ""Amelie,"" ""Run Lola Run,"" ""El Mariachi,"" ""The Good Girl,"" ""Pi,"" ""Swingers"" and many others.",My Big Fat Independent Movie,0.60,0.89,"[{""id"": 35, ""name"": ""Comedy""}]"
8,4455,Film from Daniel Davila,Harrison Montgomery,0.60,0.90,[]
7,4399,,The Helix... Loaded,0.59,0.90,"[{""id"": 28, ""name"": ""Action""}, {""id"": 35, ""name"": ""Comedy""}, {""id"": 878, ""name"": ""Science Fiction""}]"
2,1191,"When missile technology is used to enhance toy action figures, the toys soon begin to take their battle programming too seriously.",Small Soldiers,0.59,0.90,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {""id"": 878, ""name"": ""Science Fiction""}, {""id"": 28, ""name"": ""Action""}]"
6,4385,A look at the global culture and appeal of the LEGO building-block toys.,A LEGO Brickumentary,0.59,0.90,"[{""id"": 99, ""name"": ""Documentary""}]"


In [33]:
query="zombies"
search_result=searchFAISSIndex(movies,"id",query,index,nprobe=10,model=model,topk=10)
search_result=search_result[['id','overview','title','cosine_sim','L2_score']]
search_result

,id,overview,title,cosine_sim,L2_score
6,2509,"A small town is taken over by an alien plague, turning residents into zombies and all forms of mutant monsters.",Slither,0.60,0.90
2,1707,"A group of surviving people take refuge in a shopping center after the world has been over taken over by aggressive, flesh-eating zombies. A remake of the 1978 zombie film of the same name.",Dawn of the Dead,0.54,0.95
8,3881,Patrons locked inside of a bar are forced to fight monsters.,Feast,0.51,0.99
9,4412,"An idyllic island town is under attack by that most invasive of pests: zombies! Port Gamble is being overrun with braineaters, and the people seem powerless to stave them off. But wait, a rag tag band of rebels is trying to turn the tide and push the invading hordes of undead back.",ZMD: Zombies of Mass Destruction,0.50,1.00
1,1567,"After a zombie becomes involved with the girlfriend of one of his victims, their romance sets in motion a sequence of events that might transform the entire lifeless world.",Warm Bodies,0.50,1.00
7,3736,"A group of people try to survive an attack of bloodthirsty zombies while trapped in a rural Pennsylvania farmhouse. Although not the first zombie film, Night of the Living Dead is the progenitor of the contemporary ""zombie apocalypse"" horror film, and it greatly influenced the modern pop-culture zombie archetype.",Night of the Living Dead,0.49,1.01
5,2439,Roar follows a family who are attacked by various African animals at the secluded home of their keeper.,Roar,0.49,1.01
0,1318,Scientists in the Antarctic are confronted by a shape-shifting alien that assumes the appearance of the people that it kills.,The Thing,0.48,1.02
4,2387,"The final chapter of George A. Romero's ""Dead Trilogy"". In an underground government installation they are searching for a cure to overcome this strange transformation into zombies. Unfortunately, the zombies from above ground have made their way into the bunker.",Day of the Dead,0.47,1.03
3,1866,"After 23 horrifying days of gorging on human flesh, an ancient creature known as the Creeper embarks on a final voracious feeding frenzy, terrorizing a group of varsity basketball players, cheerleaders and coaches stranded on a remote highway when their bus breaks down. The terrified group is forced to come together and do battle against the winged creature hell-bent on completing its grizzly ritual.",Jeepers Creepers 2,0.47,1.03


In [34]:
query="super hero bitten by spider"
search_result=searchFAISSIndex(movies,"id",query,index,nprobe=10,model=model,topk=10)
search_result=search_result[['id','overview','title','cosine_sim','L2_score']]
search_result

,id,overview,title,cosine_sim,L2_score
1,159,"After being bitten by a genetically altered spider, nerdy high school student Peter Parker is endowed with amazing powers.",Spider-Man,0.89,0.47
7,2509,"A small town is taken over by an alien plague, turning residents into zombies and all forms of mutant monsters.",Slither,0.88,0.49
2,1534,"A large spider from the jungles of South America is accidentally transported in a crate with a dead body to America where it mates with a local spider. Soon after, the residents of a small California town disappear as the result of spider bites from the deadly spider offspring. It's up to a couple of doctors with the help of an insect exterminator to annihilate these eight legged freaks.",Arachnophobia,0.87,0.51
9,4513,"Investigating the mysterious deaths of a number of farm animals, vet Rack Hansen discovers that his town lies in the path of hoards of migrating tarantulas. Before he can take action, the streets are overrun by killer spiders, trapping a small group of towns folk in a remote hotel.",Kingdom of the Spiders,0.86,0.53
8,4034,A fancy garden party turns into upper class prey when a colony of killer wasps mutates into seven foot tall predators.,Stung,0.86,0.54
0,5,"The seemingly invincible Spider-Man goes up against an all-new crop of villain – including the shape-shifting Sandman. While Spider-Man’s superpowers are altered by an alien organism, his alter ego, Peter Parker, deals with nemesis Eddie Brock and also gets caught up in a love triangle.",Spider-Man 3,0.85,0.56
3,1637,"The residents of a rural mining town discover that an unfortunate chemical spill has caused hundreds of little spiders to mutate overnight to the size of SUVs. It's then up to mining engineer Chris McCormack and Sheriff Sam Parker to mobilize an eclectic group of townspeople, including the Sheriff's young son, Mike, her daughter, Ashley, and paranoid radio announcer Harlan, into battle against the bloodthirsty eight-legged beasts.",Eight Legged Freaks,0.84,0.56
4,1707,"A group of surviving people take refuge in a shopping center after the world has been over taken over by aggressive, flesh-eating zombies. A remake of the 1978 zombie film of the same name.",Dawn of the Dead,0.84,0.57
5,1866,"After 23 horrifying days of gorging on human flesh, an ancient creature known as the Creeper embarks on a final voracious feeding frenzy, terrorizing a group of varsity basketball players, cheerleaders and coaches stranded on a remote highway when their bus breaks down. The terrified group is forced to come together and do battle against the winged creature hell-bent on completing its grizzly ritual.",Jeepers Creepers 2,0.83,0.59
6,2387,"The final chapter of George A. Romero's ""Dead Trilogy"". In an underground government installation they are searching for a cure to overcome this strange transformation into zombies. Unfortunately, the zombies from above ground have made their way into the bunker.",Day of the Dead,0.83,0.59
